In [34]:
import scipy
import os
import bcolz
import pandas as pd
from tqdm import tqdm_notebook
import numpy as np

In [35]:

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()


def load_array(fname):
    return bcolz.open(fname)[:]


In [36]:
best207 = pd.read_csv("data/bestof207.csv")

In [3]:
oli_res_preds = load_array("data/raw_preds_224x224_blend_2resnets_5a_n_4a_ft_0.92905_armin_ordering.dat/")
oli_res_thres = load_array("data/resnet_thresholds_simple_aug_10xtta.dat/")

In [4]:
oli_etr_preds = load_array("data/etr_test_preds_raw.dat/")


In [5]:
y_train = []

df_train = pd.read_csv('data/train_v2.csv')
df_test = pd.read_csv('data/sample_submission_v2.csv')

# flatten = lambda l: [item for sublist in l for item in sublist]
# labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'cultivation',
 'artisinal_mine',
 'haze',
 'primary',
 'slash_burn',
 'habitation',
 'clear',
 'road',
 'selective_logging',
 'partly_cloudy',
 'agriculture',
 'water',
 'cloudy']

label_map = {'agriculture': 14,
 'artisinal_mine': 5,
 'bare_ground': 1,
 'blooming': 3,
 'blow_down': 0,
 'clear': 10,
 'cloudy': 16,
 'conventional_mine': 2,
 'cultivation': 4,
 'habitation': 9,
 'haze': 6,
 'partly_cloudy': 13,
 'primary': 7,
 'road': 11,
 'selective_logging': 12,
 'slash_burn': 8,
 'water': 15}

In [6]:
result = pd.DataFrame(oli_etr_preds, columns = labels)
result.head(1)

,blow_down,bare_ground,conventional_mine,blooming,cultivation,artisinal_mine,haze,primary,slash_burn,habitation,clear,road,selective_logging,partly_cloudy,agriculture,water,cloudy
0,0.02535,0.012565,0.0,0.241257,0.030148,0.0,0.020232,1.0,0.0,0.050083,0.969721,0.025549,0.085725,0.010046,0.075645,0.065289,0.0


In [7]:
new_order = {'slash_burn': 15, 'selective_logging': 14, 
             'cultivation': 8, 'clear': 5, 'habitation': 9,
             'conventional_mine': 7, 'cloudy': 6, 'primary': 12, 
             'water': 16, 'haze': 10, 'artisinal_mine': 1, 
             'partly_cloudy': 11, 'blooming': 3, 'bare_ground': 2, 
             'blow_down': 4, 'agriculture': 0, 'road': 13}

In [8]:
inv_map = {v: k for k, v in new_order.iteritems()}
inv_map.values()

['agriculture',
 'artisinal_mine',
 'bare_ground',
 'blooming',
 'blow_down',
 'clear',
 'cloudy',
 'conventional_mine',
 'cultivation',
 'habitation',
 'haze',
 'partly_cloudy',
 'primary',
 'road',
 'selective_logging',
 'slash_burn',
 'water']

In [9]:
df_neworder = result[inv_map.values()]

In [10]:
df_neworder.shape

(61191, 17)

In [ ]:
import numpy as np 

save_array("data/armin_tifetr_raw_qin_ordering.dat",
           (df_neworder.values).astype(np.int8))

In [104]:
df_neworder.to_csv("data/raw_preds_224x224_tifetr_qin_ordering.csv",index=False)

In [105]:
df_neworder.values

array([[  7.56446108e-02,   0.00000000e+00,   1.25654209e-02, ...,
          8.57254061e-02,   0.00000000e+00,   6.52889843e-02],
       [  1.80410935e-02,   0.00000000e+00,   6.06775888e-05, ...,
          2.86216748e-03,   0.00000000e+00,   2.60522143e-03],
       [  5.86430227e-02,   0.00000000e+00,   0.00000000e+00, ...,
          6.68355248e-06,   0.00000000e+00,   1.80503213e-01],
       ..., 
       [  6.32517010e-02,   0.00000000e+00,   5.12698560e-03, ...,
          3.13623357e-02,   0.00000000e+00,   1.17923373e-01],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  2.60437500e-01,   0.00000000e+00,   7.00625000e-02, ...,
          0.00000000e+00,   0.00000000e+00,   8.51875000e-02]])

### load xgb and etr 0-1 preds 

In [75]:
etr_01 = load_array("data/etr_test_preds_01.dat/")
etr_01 = etr_01[:,new_order.values()]
save_array("data/etr_test_preds_01_qin_order.dat/",etr_01)


In [69]:
etr_01.sum()

211762

In [72]:
xgb_raw = load_array("data/xgb_test_preds_raw.dat/")
xgb_raw

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [68]:
xgb_01.sum()

0

In [ ]:
xgb_01 = xgb_01[:,new_order.values()]

### load inception adn xception 

In [25]:
new_order = {'slash_burn': 15, 'selective_logging': 14, 
             'cultivation': 8, 'clear': 5, 'habitation': 9,
             'conventional_mine': 7, 'cloudy': 6, 'primary': 12, 
             'water': 16, 'haze': 10, 'artisinal_mine': 1, 
             'partly_cloudy': 11, 'blooming': 3, 'bare_ground': 2, 
             'blow_down': 4, 'agriculture': 0, 'road': 13}

inv_map = {v: k for k, v in new_order.iteritems()}
inv_map = inv_map.values()

In [17]:
def extract_01(df_test):
    y_test = []

    for f, tags in tqdm_notebook(df_test.values, miniters=1000):
        targets = np.zeros(17)

        for t in tags.split(' '):
            targets[label_map[t]] = 1 


        y_test.append(targets)

    y_test = np.array(y_test).astype(np.uint8)    
    return y_test

In [27]:
inv_map

['agriculture',
 'artisinal_mine',
 'bare_ground',
 'blooming',
 'blow_down',
 'clear',
 'cloudy',
 'conventional_mine',
 'cultivation',
 'habitation',
 'haze',
 'partly_cloudy',
 'primary',
 'road',
 'selective_logging',
 'slash_burn',
 'water']

In [32]:
xception = pd.read_csv("data/xception_full_112x112_fullnetft_10xtta.csv")

result = pd.DataFrame(extract_01(xception), columns = labels)
result[inv_map]
result.to_csv("data/xception_full_112x112_fullnetft_01_qin_ordering.csv",index=False)

In [33]:
inception = pd.read_csv("data/inception_full_150x150_10tta_augv2_150x150_mix7.csv")

result = pd.DataFrame(extract_01(inception), columns = labels)
result[inv_map]
result.to_csv("data/inception_full_150x150_10tta_augv2_01_qin_ordering.csv",index=False)